# Compute Shader

In this tutorial, we learn how to run simple compute shaders.

We start by importing `sgl` and `numpy`.

In [1]:
import sgl
import numpy as np

Next, we create a `Device` instance.
This object is used for creating and managing resources on the GPU.

In [2]:
device = sgl.Device()

Most objects in `sgl` will display useful information when being printed:

In [3]:
print(device)

Device(
  type = d3d12,
  enable_debug_layers = false,
  adapter_luid = null,
  supported_shader_model = sm_6_7
  shader_cache_enabled = false
  shader_cache_path = ""
)


At a glance we can see what type of underlying graphics API is being used, if debug layers are enabled, the default shader model being used etc.

Next, we write a simple slang compute kernel that adds two floating point arrays. We mark our shader entry point using the `[[shader("compute")]]` attribute. This will allow the slang compiler to find the entry point by name.

[comment]: <> (embed compute_shader.slang)
```C#
// compute_shader.slang

StructuredBuffer<float> a;
StructuredBuffer<float> b;
RWStructuredBuffer<float> c;

[shader("compute")]
[numthreads(32, 1, 1)]
void main(uint3 tid : SV_DispatchThreadID)
{
    c[tid.x] = a[tid.x] + b[tid.x];
}
```


We can load the shader code as a slang shader module using `load_module_from_source`.
Once we have the module loaded, we can create a new compute kernel for our `main` entry point in the module.

In [4]:
program = device.load_program(module_name="compute_shader.slang", entry_point_names=["main"])
kernel = device.create_compute_kernel(program=program)

We continue to create buffers to pass to our compute shader. Buffers `a` and `b` will be used as input only, while buffer `c` will be used as an output.
We create all three buffers as _structured buffers_, using the kernels reflection data to determine the size of each element in the buffer.
Buffers `a` and `b` are initialized with linear sequences using `numpy.linspace`.
Buffer `c` is not initialized, but we have to set its `usage` to `sgl.ResourceUsage.unordered_access` in order to allow GPU side writes.


In [5]:
buffer_a = device.create_structured_buffer(
    element_count=1024,
    struct_type=kernel.reflection.a,
    data=np.linspace(0, 1, 1024, dtype=np.float32),
)
buffer_b = device.create_structured_buffer(
    element_count=1024,
    struct_type=kernel.reflection.b,
    data=np.linspace(1, 0, 1024, dtype=np.float32),
)
buffer_c = device.create_structured_buffer(
    element_count=1024,
    struct_type=kernel.reflection.c,
    usage=sgl.ResourceUsage.unordered_access,
)

We can now dispatch the compute kernel. We first specify the number of threads to run using `thread_count=[1024, 1, 1]`. This will automatically be converted to a number of thread groups to run based on the thread group size specified in the shader (`[numthreads(32,1,1)]`). We bind our buffers as global resources using `vars={...}`.

In [6]:
kernel.dispatch(thread_count=[1024, 1, 1], vars={"a": buffer_a, "b": buffer_b, "c": buffer_c})

After the dispatch, we can read back the contents of the `c` buffer to a numpy array and print it.

In [7]:
data = buffer_c.to_numpy().view(np.float32)
print(data)
assert np.all(data == 1.0)

[1. 1. 1. ... 1. 1. 1.]
